In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('2-fft-malicious-n-0-3-m-1-data.csv')
df2 = pd.read_csv('2-fft-malicious-n-0-5-m-9-data.csv')
df3 = pd.read_csv('2-fft-malicious-n-0-10-m-14-data.csv')
df4 = pd.read_csv('2-fft-malicious-n-0-12-m-9-data.csv')
df5 = pd.read_csv('2-fft-malicious-n-0-15-m-1-data.csv')

In [3]:
df1 = df1.loc[df1['router'] == 1]
df2 = df2.loc[df2['router'] == 1]
df3 = df3.loc[df3['router'] == 1]
df4 = df4.loc[df4['router'] == 1]
df5 = df5.loc[df5['router'] == 1]

In [4]:
df1 = df1.drop(columns=['router'])
df2 = df2.drop(columns=['router'])
df3 = df3.drop(columns=['router'])
df4 = df4.drop(columns=['router'])
df5 = df5.drop(columns=['router'])

In [5]:
df1.to_csv('2-fft-normal-n-0-3-data-r1-mal.csv',index=False)
df2.to_csv('2-fft-normal-n-0-5-data-r1-mal.csv',index=False)
df3.to_csv('2-fft-normal-n-0-10-data-r1-mal.csv',index=False)
df4.to_csv('2-fft-normal-n-0-12-data-r1-mal.csv',index=False)
df5.to_csv('2-fft-normal-n-0-15-data-r1-mal.csv',index=False)

In [6]:
df1 = pd.read_csv('2-fft-normal-n-0-3-data-r1-mal.csv')
df2 = pd.read_csv('2-fft-normal-n-0-5-data-r1-mal.csv')
df3 = pd.read_csv('2-fft-normal-n-0-10-data-r1-mal.csv')
df4 = pd.read_csv('2-fft-normal-n-0-12-data-r1-mal.csv')
df5 = pd.read_csv('2-fft-normal-n-0-15-data-r1-mal.csv')

In [7]:
def timecount(df):
    timearr = []
    interval = 99
    count = 0
    for index, row in df.iterrows():
        if row["timestamp"]<=interval:
            count+=1
        else:
            timearr.append([interval+1,count])
            count=1
            interval+=100
    timearr.append([interval+1,count])
    return timearr

In [8]:
def maxcount(timearr,df):
    countarr = []
    increarr = []
    maxarr = []
    for i in range(len(timearr)):
        for cnt in range(timearr[i][1],0,-1):
            countarr.append(cnt)
            maxarr.append(timearr[i][1])
            increment = timearr[i][1] - cnt + 1
            increarr.append(increment)
    df = df.assign(packet_count_decr=countarr)
    df = df.assign(packet_count_incr=increarr)
    df = df.assign(max_packet_count=maxarr)
    return df

In [ ]:
df1 = maxcount(timecount(df1),df1)
df2 = maxcount(timecount(df2),df2)
df3 = maxcount(timecount(df3),df3)
df4 = maxcount(timecount(df4),df4)
df5 = maxcount(timecount(df5),df5)

In [ ]:
df1

In [ ]:
def rename(df):
    df["packet_count_index"] = df["packet_count_decr"]*df["packet_count_incr"]
    df["packet_max_index"] = df["packet_count_index"]*df["max_packet_count"]
    df["port_index"] = df["outport"]*df["inport"]
    df["cache_coherence_flit_index"] = df["cache_coherence_type"]*df["flit_id"]
    df["flit_index"] = df["cache_coherence_flit_index"]*df["flit_type"]
    df["traversal_index"] = df["flit_index"]*df["traversal_id"]
    df["cache_coherence_vnet_index"] = df["cache_coherence_type"]*df["vnet"]
    df["vnet_vc_index"] = df["vnet"]*df["vc"]
    df["vnet_vc_cc_index"] = df["vnet"]*df["cache_coherence_vnet_index"]
    df.head(50)

In [ ]:
rename(df1)
rename(df2)
rename(df3)
rename(df4)
rename(df5)

In [ ]:
df1['target'] = 0
df2['target'] = 0
df3['target'] = 0
df4['target'] = 0
df5['target'] = 0

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

In [ ]:
df4

In [ ]:
df5

In [ ]:
df1.dtypes

In [82]:
df1.to_csv('2-fft-normal-n-0-3-data-r1-mal.csv',index=False)
df2.to_csv('2-fft-normal-n-0-5-data-r1-mal.csv',index=False)
df3.to_csv('2-fft-normal-n-0-10-data-r1-mal.csv',index=False)
df4.to_csv('2-fft-normal-n-0-12-data-r1-mal.csv',index=False)
df5.to_csv('2-fft-normal-n-0-15-data-r1-mal.csv',index=False)